# NLE Assessed Coursework 3: Question 1

For this assessment, you are expected to complete and submit 4 notebook files.  There is 1 notebook file for each question (to speed up load times).  This is notebook 1 out of 4.

Marking guidelines are provided as a separate document.

In order to provide unique datasets for analysis by different students, you must enter your candidate number in the following cell.

In [1]:
candidateno=203068 #this MUST be updated to your candidate number so that you get a unique data sample


In [2]:
#preliminary imports
import sys
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
sys.path.append(r'/Users/danielsimons/Documents/OneDrive/Computer Science/Year 2/NLE/resources')

import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import zip_longest
from nltk.tokenize import word_tokenize

from sussex_nltk.corpus_readers import AmazonReviewCorpusReader
import random
from nltk.corpus import stopwords
import spacy
nlp=spacy.load('en_core_web_sm')
from nltk.corpus import gutenberg
import nltk
nltk.download('gutenberg')

Sussex NLTK root directory is /Users/danielsimons/Documents/OneDrive/Computer Science/Year 2/NLE/resources


[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/danielsimons/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

## Question 1: Part-of-Speech Tagging (25 marks)


In [3]:
#Do NOT change the code in this cell.

#preparing corpus
def display_sent(asent):
    headings=["token","lower","lemma","pos","NER"]
    info=[]
    for t in asent:
        info.append([t.text,t.lower_,t.lemma_,t.pos_,t.ent_type_])
    return(pd.DataFrame(info,columns=headings))

def tag_sent(asent):
    tagged=[]
    for t in asent:
        tagged.append((t.lower_,t.pos_))
    return tagged

def clean_text(astring):
    #replace newlines with space
    newstring=re.sub("\n"," ",astring)
    #remove title and chapter headings
    newstring=re.sub("\[[^\]]*\]"," ",newstring)
    newstring=re.sub("VOLUME \S+"," ",newstring)
    newstring=re.sub("CHAPTER \S+"," ",newstring)
    newstring=re.sub("\s\s+"," ",newstring)
    #return re.sub("([^\.|^ ])  +",r"\1 .  ",newstring).lstrip().rstrip()
    return newstring.lstrip().rstrip()

def display_tags(sentslist):
    taglist={}
    for s in sentslist:
        for t in s:
            form=t.lower_
            pos=t.pos_
            taglist[pos]=taglist.get(pos,0)+1
    print(len(taglist.keys()))
    print(taglist)
        
def get_train_test(sentslist,seed=candidateno):
    random.seed(seed)
    random.shuffle(sentslist)
    testsize=10
    train=[tag_sent(s) for s in sentslist[testsize:]]
    test=[tag_sent(s) for s in sentslist[:testsize]]
    return train,test
    
alice=clean_text(gutenberg.raw('carroll-alice.txt'))
nlp_alice=list(nlp(alice).sents)

The code below will generate (unique to you) training and test sets of pos-tagged sentences from the novel "Alice in Wonderland"

In [4]:
#do not change the code in this cell
allsents=list(nlp_alice)
train,test=get_train_test(allsents)

In [5]:
print(train)

[[('asked', 'VERB'), ('alice', 'PROPN'), ('again', 'ADV'), (',', 'PUNCT'), ('in', 'ADP'), ('a', 'DET'), ('louder', 'ADJ'), ('tone', 'NOUN'), ('.', 'PUNCT'), ("'", 'PUNCT')], [('she', 'PRON'), ('was', 'AUX'), ('a', 'DET'), ('good', 'ADJ'), ('deal', 'NOUN'), ('frightened', 'VERB'), ('by', 'ADP'), ('this', 'DET'), ('very', 'ADV'), ('sudden', 'ADJ'), ('change', 'NOUN'), (',', 'PUNCT'), ('but', 'CCONJ'), ('she', 'PRON'), ('felt', 'VERB'), ('that', 'SCONJ'), ('there', 'PRON'), ('was', 'AUX'), ('no', 'DET'), ('time', 'NOUN'), ('to', 'PART'), ('be', 'AUX'), ('lost', 'VERB'), (',', 'PUNCT'), ('as', 'SCONJ'), ('she', 'PRON'), ('was', 'AUX'), ('shrinking', 'VERB'), ('rapidly', 'ADV'), (';', 'PUNCT'), ('so', 'ADV'), ('she', 'PRON'), ('set', 'VERB'), ('to', 'PART'), ('work', 'VERB'), ('at', 'ADP'), ('once', 'ADV'), ('to', 'PART'), ('eat', 'VERB'), ('some', 'DET'), ('of', 'ADP'), ('the', 'DET'), ('other', 'ADJ'), ('bit', 'NOUN'), ('.', 'PUNCT')], [('in', 'ADP'), ('my', 'DET'), ('youth', 'NOUN'), (',

In the cell below, a class `unigram_tagger` is defined, which will be used in parts a and b of this question

In [6]:
#do not change the code in this cell
class unigram_tagger():
    
    def __init__(self,trainingdata=[]):
        self.tagcounts={}
        self.wordcounts={}
        self.tagperwordcounts={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            for token,tag in sentence:
                self.tagcounts[tag]=self.tagcounts.get(tag,0)+1
                self.wordcounts[token]=self.wordcounts.get(token,0)+1
                current=self.tagperwordcounts.get(token,{})
                current[tag]=current.get(tag,0)+1
                self.tagperwordcounts[token]=current
                

a) Create an instance of the `unigram_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).
Explain what is stored, after training, in the three instance variables:
* `self.tagcounts`
* `self.wordcounts`
* `self.tagperwordcounts`

You should refer to the code and specific examples from the training data. \[4 marks \]

In [7]:
u = unigram_tagger(train)

In [8]:
print(u.tagcounts)

{'VERB': 4606, 'PROPN': 1485, 'ADV': 2261, 'PUNCT': 7151, 'ADP': 2601, 'DET': 3547, 'ADJ': 1381, 'NOUN': 3302, 'PRON': 3220, 'AUX': 1621, 'CCONJ': 1112, 'SCONJ': 672, 'PART': 843, 'NUM': 199, 'X': 8, 'INTJ': 149}


print(u.wordcounts)

In [9]:
print(u.wordcounts)

{'asked': 17, 'alice': 394, 'again': 80, ',': 2392, 'in': 366, 'a': 626, 'louder': 1, 'tone': 40, '.': 965, "'": 2196, 'she': 550, 'was': 365, 'good': 27, 'deal': 12, 'frightened': 7, 'by': 58, 'this': 131, 'very': 144, 'sudden': 5, 'change': 13, 'but': 165, 'felt': 23, 'that': 311, 'there': 99, 'no': 90, 'time': 71, 'to': 720, 'be': 145, 'lost': 3, 'as': 257, 'shrinking': 4, 'rapidly': 2, ';': 191, 'so': 148, 'set': 14, 'work': 8, 'at': 211, 'once': 34, 'eat': 18, 'some': 51, 'of': 512, 'the': 1615, 'other': 38, 'bit': 16, 'my': 58, 'youth': 5, 'father': 6, 'william': 8, 'replied': 29, 'his': 95, 'son': 1, 'i': 530, 'feared': 1, 'it': 589, 'might': 28, 'injure': 1, 'brain': 1, 'now': 59, "'m": 54, 'perfectly': 4, 'sure': 23, 'have': 88, 'none': 4, 'why': 39, 'do': 135, 'and': 858, 'thought': 74, 'well': 60, 'wait': 1, 'had': 184, 'nothing': 34, 'else': 11, 'perhaps': 17, 'after': 43, 'all': 179, 'tell': 31, 'her': 248, 'something': 18, 'worth': 4, 'hearing': 4, "'ll": 53, 'get': 46, '

In [10]:
print(u.tagperwordcounts)

{'asked': {'VERB': 17}, 'alice': {'PROPN': 394}, 'again': {'ADV': 80}, ',': {'PUNCT': 2392}, 'in': {'ADP': 363, 'ADV': 3}, 'a': {'DET': 626}, 'louder': {'ADJ': 1}, 'tone': {'NOUN': 40}, '.': {'PUNCT': 965}, "'": {'PUNCT': 2190, 'PART': 6}, 'she': {'PRON': 547, 'PROPN': 3}, 'was': {'AUX': 361, 'VERB': 4}, 'good': {'ADJ': 26, 'ADV': 1}, 'deal': {'NOUN': 12}, 'frightened': {'VERB': 2, 'ADJ': 5}, 'by': {'ADP': 57, 'ADV': 1}, 'this': {'DET': 130, 'PROPN': 1}, 'very': {'ADV': 143, 'ADJ': 1}, 'sudden': {'ADJ': 5}, 'change': {'NOUN': 4, 'VERB': 9}, 'but': {'CCONJ': 162, 'SCONJ': 3}, 'felt': {'VERB': 23}, 'that': {'SCONJ': 138, 'DET': 170, 'ADP': 2, 'ADV': 1}, 'there': {'PRON': 89, 'ADV': 10}, 'no': {'DET': 63, 'ADV': 7, 'INTJ': 20}, 'time': {'NOUN': 68, 'PROPN': 3}, 'to': {'PART': 436, 'ADP': 284}, 'be': {'AUX': 144, 'VERB': 1}, 'lost': {'VERB': 3}, 'as': {'SCONJ': 203, 'ADV': 54}, 'shrinking': {'VERB': 4}, 'rapidly': {'ADV': 2}, ';': {'PUNCT': 191}, 'so': {'ADV': 138, 'CCONJ': 3, 'SCONJ': 7},

The tag counts method shows the number of times a different type of word appears in a document. For example, in the Alice in Wonderland corpus there are 2261 adverbs as you can see from the example above.

The word counts function prints out how many times each different word appears in a corpus. For example, the word ‘asked’ appears 17 times in the corpus as you can see from the example above.

The tag per word counts function prints out the type of word that each word that appears in the corpus is and declares how many times that word appears. For example, the word ‘asked’ appears in the Alice and Wonderland corpus 17 times and is declared by the ‘tag per word counts’ function as a ‘VERB’.

b) In the cells below, **extend** the code for the `unigram_tagger` class so that it also has a `tag()` method.  This method should assign the tag, $t$, which maximises the unigram tag probability, $P(t|w)$, for the observed word, $w$.  **Evaluate** the performance of the `unigram_tagger` on your test sentences.  **Discuss** your results. \[8 marks\] 



In [11]:
class unigram_tagger(unigram_tagger):
    def tag(self):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-11-f48aad42edb2>, line 3)

In the cell below, a `hmm_tagger` class is defined, which will be used in parts c and d of this question.

In [ ]:
#do not change the code in this cell
class hmm_tagger():
    
    def __init__(self,trainingdata=[]):
        
        self.emissions={}
        self.transitions={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            previous="START"
            for token,tag in sentence:
                
                current=self.emissions.get(tag,{})
                current[token]=current.get(token,0)+1
                self.emissions[tag]=current
                bigram=self.transitions.get(previous,{})
                bigram[tag]=bigram.get(tag,0)+1
                self.transitions[previous]=bigram
                previous=tag
                
        self.emissions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.emissions.items()}
        self.transitions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.transitions.items()}

c) **Create** an instance of the `hmm_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).  With reference to your testing sentences (stored in `test`), **explain**
* how to calculate the probability of an observed sequence of words for a given sequence of tags: $$P(w_1^n|t_1^n)$$
* how to calculate the probablity of a possible sequence of tags for a given sequence of words: $$P(t_1^n|w_1^n)$$

\[6 marks\]

d) Using one of your `test` sentences as an example, **explain** how the Viterbi algorithm can be used to find the most likely tag sequence.  You do not need to write code for this question but should explain the calculations that need to be made at each step.  **Comment** on whether the sequence found by the Viterbi algorithm is correct for your chosen test sentence.  **Discuss** why using the Viterbi algorithm is better than the brute-force approach of enumerating and testing all tag sequence possibilities.\[7 marks\]
